In [20]:
!pip install -q google-adk


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [21]:
import os
try:
    from google.colab import userdata
    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set

# Website Builder Agent

Build an agent that helps plan websites from descriptions. Your agent will have these tools:

1. **generate_layout** — Plan the structure/sections of a page based on its type and description
2. **suggest_color_scheme** — Recommend a color palette based on style and industry
3. **suggest_content** — Generate copywriting text for a specific section

Tools return hardcoded/fake data — the goal is designing the **interface** between the model and your code.

## Setup

In [22]:
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.genai import types

## TODO 1: Define your tools

Just write plain Python functions. The ADK infers the schema from:
- **Type hints** → parameter types
- **Docstring** → tool description

The first tool is done for you as an example. Implement the remaining two.

In [23]:
# Example tool (complete)

def generate_layout(page_type: str, description: str) -> dict:
    """Plan the layout and sections for a website page.

    Args:
        page_type: The type of page (e.g. "landing", "portfolio", "blog", "restaurant").
        description: What the page is for and any specific requirements.

    Returns:
        dict: status and a layout plan describing the page sections.
    """
    return {
        "status": "success",
        "report": (
            f"Layout plan for a {page_type} page:\n"
            f"1. Navigation bar — logo on the left, menu links on the right\n"
            f"2. Hero section — large headline, subtitle, and a call-to-action button\n"
            f"3. Features/services section — 3-column grid highlighting key offerings\n"
            f"4. Testimonials section — customer quotes in a carousel\n"
            f"5. Contact section — address, phone, email, and a simple form\n"
            f"6. Footer — social media links, copyright notice\n\n"
            f"Design notes based on description: {description}"
        ),
    }


def suggest_color_scheme(style: str, industry: str) -> dict:
    """Suggest a color palette for a website based on the desired style and industry.

    Args:
        style: The visual style (e.g. "professional", "playful", "dark", "minimal", "bold").
        industry: The business type (e.g. "restaurant", "tech startup", "law firm", "photography").

    Returns:
        dict: status and a recommended color palette.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: return a hardcoded color palette as plain text, e.g.:
    #   "Primary: deep navy (#1a2b4a), Accent: warm gold (#d4a843),
    #    Background: off-white (#f9f6f0), Text: charcoal (#2d2d2d)"
    # You can vary the palette based on style or industry if you want!
    pass


def suggest_content(section_type: str, business_name: str) -> dict:
    """Generate suggested copywriting text for a website section.

    Args:
        section_type: The section to write for (e.g. "hero", "about", "testimonials", "call_to_action").
        business_name: The name of the business the website is for.

    Returns:
        dict: status and suggested text content for the section.
    """
    # TODO: implement — return a dict with "status" and "report" keys
    # Hint: return hardcoded marketing copy, e.g. for a "hero" section:
    #   "Headline: Welcome to {business_name}
    #    Subtitle: We bring your ideas to life.
    #    Button text: Get Started"
    pass

## TODO 2: Define your agent

In [ ]:
agent = Agent(
    model="gemini-3-flash-preview",
    name="website_builder",
    # TODO: Write an instruction that gives the agent a web-designer persona.
    # Hint: describe what the agent can help with, its tone, and any guidelines.
    # The agent should use tools to gather layout, colors, and content, then
    # combine everything into a final HTML page.
    instruction="""You are a website builder assistant. You help users create websites.

Your workflow:
1. Use generate_layout to plan the page structure
2. Use suggest_color_scheme to get a color palette
3. Use suggest_content to get copywriting for each section
4. Combine all the tool outputs into a complete, styled HTML page as your final response

Always output a full HTML page at the end. The HTML should incorporate the layout, colors, and content from your tools.
""",
    tools=[generate_layout, suggest_color_scheme, suggest_content],
)

## Run your agent

Once you've filled in the TODOs above, run this cell to test your agent!

In [25]:
runner = InMemoryRunner(agent=agent, app_name="website_builder")
runner.auto_create_session = True
user_content = types.Content(role="user", parts=[types.Part.from_text(text="Plan a landing page for a coffee shop called Bean & Brew. I want a warm, cozy vibe with a hero section and contact info.")])

final_text = ""
async for event in runner.run_async(user_id="user1", session_id="session1", new_message=user_content):
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.function_call:
                print(f"🔧 Tool call: {part.function_call.name}({dict(part.function_call.args)})")
            if part.function_response:
                print(f"  ↳ Response: {part.function_response.name} returned")
            if part.text:
                final_text = part.text

print(f"\n{'='*60}\nFinal response:\n{'='*60}\n{final_text}")

🔧 Tool call: generate_layout({'description': "A warm and cozy landing page for a coffee shop featuring a hero section, an 'about our beans' section, a featured menu preview, and contact information.", 'page_type': 'landing'})
🔧 Tool call: suggest_color_scheme({'industry': 'coffee shop', 'style': 'warm and cozy'})
🔧 Tool call: suggest_content({'business_name': 'Bean & Brew', 'section_type': 'hero'})
🔧 Tool call: suggest_content({'business_name': 'Bean & Brew', 'section_type': 'contact_info'})
  ↳ Response: generate_layout returned
  ↳ Response: suggest_color_scheme returned
  ↳ Response: suggest_content returned
  ↳ Response: suggest_content returned
🔧 Tool call: suggest_color_scheme({'industry': 'restaurant', 'style': 'warm'})
🔧 Tool call: suggest_content({'business_name': 'Bean & Brew', 'section_type': 'hero'})
🔧 Tool call: suggest_content({'business_name': 'Bean & Brew', 'section_type': 'about'})
  ↳ Response: suggest_color_scheme returned
  ↳ Response: suggest_content returned
  ↳ R

## Tips

- **Docstrings matter** — the ADK uses them as tool descriptions. Be specific about what each parameter means.
- **Fake data is fine** — the goal is designing the agent's interface, not building a real website.
- **Test edge cases** — what happens when a user asks for a page type your tools don't handle?

### Stretch Goals
- Add a `suggest_images` tool that recommends stock photo descriptions for each section
- Support different page types with different layouts (e.g. portfolio vs. blog)
- Add a `generate_sitemap` tool that plans a multi-page website structure